# Generating the disruption network
## And calculating the disruption index

One of the most important steps in studying the effect of disruption in songs is actually calculating it based on the similarity matrix we obtained before. That way we use the same list of songs that was used to generate the similarity matrix and use to calculate the disruption score for each song in our dataset. That means that after calculating we can find who were the songs most disruptive in our dataset based on its metadata.

## The code below generates a songs disruption network using:
1. list of songs (in the same order as the similarity matrix)
2. A similarity matrix (ordered by release) so we know that the next song i + 1 is a song released after i
3. A similarity threshold that will determine if there is an edge between nodes

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import collections

from tqdm import tqdm
from networkx.drawing.nx_pylab import draw_networkx

In [2]:
def generate_network(list_of_songs, similarity_matrix, similarity_threshold=0.80):
    """ Generate a disruption network based on: 
    If a song has a similarity with another over the threshold than an edge is made to connect both of them.

    Args:
        1. list of songs (in the same order as the similarity matrix)
        2. A similarity matrix (ordered by release) so we know that the next song i + 1 is a song released after i
        3. A similarity threshold that will determine if there is an edge between nodes
    Returns:
        A network as a networkx.DiGraph Object
    """
    slice_index = 0
    G = nx.DiGraph()

    for i in tqdm(range(len(list_of_songs))):
        edge_count = 0
        
        G.add_node(i)
        
        for j in range(i + 1, len(list_of_songs)):
            # If there is a high similarity, create an edge between the nodes
            if similarity_matrix[i][j] > similarity_threshold:
                G.add_edge(j, i)
                edge_count += 1
        
        # If this node does not have a similarity with any other node, then remove the node
        if edge_count < 1:
            G.remove_node(i)

    return G

def get_disruption_index_for_nodes(list_of_songs, graph):
    """ Compute the actual disruption indexes for the graph based on the nodes(songs) and its
    connections (if its influenced or if it influenced another song) """
    disruption_info = {}

    for i in tqdm(range(len(list_of_songs))):    
        if graph.has_node(i):
            songs_after = range(i + 1, len(list_of_songs))
            song_influences = [edge[1] for edge in graph.edges(i) if edge[1] != i]

            ni = 0
            nj = 0
            nk = 0
            
            for song_after in songs_after:
                consolidating_influence = False
                if graph.has_edge(song_after , i):
                
                    for influence in song_influences:
                        if graph.has_edge(song_after, influence):
                            consolidating_influence = True
                            break
                
                    if consolidating_influence:
                        nj += 1
                    else:
                        ni += 1
                
                else:
                    for influence in song_influences:
                        if graph.has_edge(song_after, influence):
                            nk += 1
    
            disruption_info[list_of_songs.iloc[i]['id']] = [ni, nj, nk, float((ni-nj)) / float((ni+nj+nk))] if (ni + nj + nk) > 0 else [ni, nj, nk, 0]
    
    return disruption_info


## Loading our features, matrix and dataframe
The first step to build the network it is to load the files generated in prior steps

### `Information about the dataset:`
 - The filtered dataset refers removed the songs that their mp3 had no sound.

### `Information about the feature vectors:`
 -  Features can be both calculated using the MFCC or the concatenation of the features from the transfer learning convnet


### `Information about the similarity matrix:`

Similarity matrix can have more or less similarities between songs based on the gamma value
 - Higher means it is more strict
 - Lesser means that songs that are different will be deemed as similar

In [4]:
from pathlib import Path

DATASET_PATH = Path("../dataset")

def load_npy(file_path):
    print(f"Loading: {file_path} ...")
    return np.load(file_path)

def load_features_from_file(file_name, feature_type):
    if feature_type == "mfcc":
        return load_npy(DATASET_PATH / "input" / "feature_vectors" / "mfcc" / file_name)
    elif feature_type == "transfer_learning":
        return load_npy(DATASET_PATH / "input" / "feature_vectors" / "transfer_learning" / file_name)
    else:
        raise TypeError("This feature type is not supported")

def load_similarity_matrix(file_name, feature_type):
    if feature_type == "mfcc":
        return load_npy(DATASET_PATH / "input" / "similarity_matrices" / "mfcc" / file_name)
    elif feature_type == "transfer_learning":
        return load_npy(DATASET_PATH / "input" / "similarity_matrices" / "transfer_learning" / file_name)
    else:
        raise TypeError("This feature type is not supported")

def load_dataframe(dataframe_file):
    return pd.read_csv(DATASET_PATH / "input" / "csvs" / dataframe_file)

feat_type = "transfer_learning"
datset_size = 30001
gamma = 0.1

DF_PATH = f"sorted_song_info_{datset_size}.csv"
FEATS_PATH = f"{feat_type}_feature_vector_{datset_size}_samples.npy"
SIMILARITY_MATRIX_PATH = f"{feat_type}_{datset_size}_samples_{gamma}_gamma.npy"

dataframe = load_dataframe(DF_PATH)
features = load_features_from_file(FEATS_PATH, feat_type)
similarity_matrix = load_similarity_matrix(SIMILARITY_MATRIX_PATH, feat_type)

Loading: ../dataset/input/feature_vectors/transfer_learning/transfer_learning_feature_vector_30001_samples.npy ...
Loading: ../dataset/input/similarity_matrices/transfer_learning/transfer_learning_30001_samples_0.1_gamma.npy ...


Now we can call the functions defined above to generate the network and calculate the disruption index!

In [6]:
dataframe.describe()

,Unnamed: 0,index,popularity,release,danceability,energy,key,mode,valence,tempo,duration_ms,mapping_to_fv_index
count,30001.000000,30001.000000,30001.000000,30001.000000,30001.000000,30001.000000,30001.000000,30001.000000,30001.000000,30001.000000,3.000100e+04,30001.000000
mean,15000.000000,54342.161628,35.382421,1987.636412,0.508706,0.632923,5.267524,0.674944,0.505589,121.193937,2.497593e+05,54686.167694
std,8660.687049,31444.376225,14.958805,11.590496,0.173874,0.250835,3.549276,0.468404,0.263471,28.710637,1.037787e+05,31656.246802
min,0.000000,2.000000,0.000000,1923.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.018700e+04,8.000000
25%,7500.000000,27008.000000,25.000000,1979.000000,0.382000,0.453000,2.000000,0.000000,0.287000,99.766000,1.912000e+05,27178.000000
50%,15000.000000,54309.000000,34.000000,1991.000000,0.516000,0.669000,5.000000,1.000000,0.507000,119.829000,2.365830e+05,54602.000000
75%,22500.000000,81532.000000,46.000000,1997.000000,0.637000,0.851000,9.000000,1.000000,0.724000,138.209000,2.866670e+05,82236.000000
max,30000.000000,109172.000000,80.000000,2002.000000,0.988000,1.000000,11.000000,1.000000,0.998000,242.903000,2.703227e+06,109267.000000


In [7]:
graph = generate_network(dataframe, similarity_matrix)

100%|██████████| 30001/30001 [35:53<00:00, 13.93it/s] 


# Exporting the graph generated

Here we export the generated graph in a way we can analyse it on Gephi later

In [8]:
disruption_index = get_disruption_index_for_nodes(dataframe, graph)

100%|██████████| 30001/30001 [5:14:14<00:00,  1.59it/s] 


In [16]:
nx.write_gexf(graph, DATASET_PATH / "output" / "graphs" / f"{feat_type}_{len(disruption_index)}_{gamma}.gexf")

Some of 29000+ songs used to build the network had no connection with any other, so they didn't even enter the network:

```
# If this node does not have a similarity with any other node, then remove the node
if edge_count < 1:
    G.remove_node(i)
```

That is why we have only 26091 with a disruption index

In [17]:
len(disruption_index)

26638

# Exporting the disruption index

In [18]:
import pickle

In [19]:
# Store data (serialize)
with open(DATASET_PATH / "output" / "disruption_index" / f'{feat_type}_{len(disruption_index)}_{gamma}.pickle', 'wb') as handle:
    pickle.dump(disruption_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

### We should store as a dataframe too

In [20]:
with open(DATASET_PATH / "output" / "disruption_index" / f'{feat_type}_{len(disruption_index)}_{gamma}.pickle', 'rb') as handle:
    loaded_disruption_index = pickle.load(handle)


### Generating the dataframe with the disruption index

In [21]:
disruption_index_df = pd.DataFrame(loaded_disruption_index).T
disruption_index_df.reset_index(inplace=True)
disruption_index_df.columns = ['id', 'ni', 'nj', 'nk', 'disruption']
disruption_index_df.head()

,id,ni,nj,nk,disruption
0,oXEbiIPUippqpNjc,2.0,0.0,0.0,1.0
1,QjxUmiXnnXxYDt72,19.0,0.0,0.0,1.0
2,c94tTSauKXFhFJDI,33.0,0.0,0.0,1.0
3,3MEb9LZbB80nQ1a8,68.0,0.0,0.0,1.0
4,WmoFpVkLhArKSfrZ,17.0,0.0,0.0,1.0


Joining song info and song disruption datasets 

In [22]:
song_info_with_disruption = pd.merge(disruption_index_df, dataframe, on='id')
song_info_with_disruption.head()

,id,ni,nj,nk,disruption,Unnamed: 0,index,artist,song,album_name,...,popularity,release,danceability,energy,key,mode,valence,tempo,duration_ms,mapping_to_fv_index
0,oXEbiIPUippqpNjc,2.0,0.0,0.0,1.0,0,88900,Bessie Smith,Nobody Knows You When You're Down and Out,The Best of Bessie Smith,...,43.0,1923,0.614,0.0423,4.0,1.0,0.211,89.822,177133,6465
1,QjxUmiXnnXxYDt72,19.0,0.0,0.0,1.0,1,47168,Fats Waller,Ain't Misbehavin',"If You Got To Ask, You Ain't Got It!",...,37.0,1926,0.515,0.2220,0.0,0.0,0.350,98.358,237773,69082
2,c94tTSauKXFhFJDI,33.0,0.0,0.0,1.0,2,67033,Blind Willie Johnson,Let Your Light Shine On Me,Dark Was The Night (Mojo Workin'- Blues For Th...,...,24.0,1928,0.439,0.2150,7.0,1.0,0.440,165.549,188373,51783
3,3MEb9LZbB80nQ1a8,68.0,0.0,0.0,1.0,3,5986,Louis Armstrong,St. James Infirmary,The Complete Hot Five And Hot Seven Recordings...,...,29.0,1928,0.693,0.1820,5.0,0.0,0.588,116.508,191867,71045
4,WmoFpVkLhArKSfrZ,17.0,0.0,0.0,1.0,8,57878,Billie Holiday,These Foolish Things (Remind Me Of You),Lady Day: The Complete Billie Holiday On Colum...,...,28.0,1933,0.582,0.2160,7.0,1.0,0.531,95.463,197693,12949


In [23]:
song_info_with_disruption.to_csv(DATASET_PATH / "output" / "csv_with_disruption" / f"song_info_with_disruption_{len(song_info_with_disruption)}_feat_{feat_type}_gamma_{gamma}.csv", index=False)